In [12]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

### ● 不動産の一つの物件から情報を取得する。

In [13]:
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13111&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="cassetteitem")

In [14]:
# スクレイピングした HTML情報出力
with open("test.txt", "w", encoding="utf-8") as f:
    f.write(str(soup))

print("Items found:", items is not None)

Items found: True


In [15]:
# 各物件情報の取得
try:
    property_name = items.find(class_="cassetteitem_content-title").get_text() if items.find(class_="cassetteitem_content-title") else None
    print("物件名称 (Property Name):", property_name)
except AttributeError as e:
    print("Error finding property_name:", e)

try:
    category = items.find(class_="cassetteitem_content-label").span.get_text() if items.find(class_="cassetteitem_content-label") else None
    print("カテゴリー (Category):", category)
except AttributeError as e:
    print("Error finding category:", e)

try:
    address = items.find(class_="cassetteitem_detail-col1").get_text() if items.find(class_="cassetteitem_detail-col1") else None
    print("住所 (Address):", address)
except AttributeError as e:
    print("Error finding address:", e)

try:
    nearest_stations = [station.get_text() for station in items.find_all(class_="cassetteitem_detail-text")] if items.find_all(class_="cassetteitem_detail-text") else None
    print("最寄り駅 (Nearest Stations):", nearest_stations)
except AttributeError as e:
    print("Error finding nearest_stations:", e)

try:
    construction_info = items.find(class_="cassetteitem_detail-col3").find_all("div") if items.find(class_="cassetteitem_detail-col3") else None
    years_since_const = construction_info[0].get_text() if construction_info and len(construction_info) > 0 else None
    number_of_floors = construction_info[1].get_text() if construction_info and len(construction_info) > 1 else None
    print("築年数 (Years Since Construction):", years_since_const)
    print("階建 (Number of Floors):", number_of_floors)
except AttributeError as e:
    print("Error finding construction_info:", e)

try:
    floor_number_td = items.select_one("tr.js-cassette_link .cassetteitem_other-col03")
    floor_number = floor_number_td.get_text(strip=True) if floor_number_td else None
    print("階数 (Floor Number):", floor_number)
except AttributeError as e:
    print("Error finding floor_number:", e)

try:
    rent_info = items.select_one(".cassetteitem_other tbody .js-cassette_link")
    rent_admin_fee = " / ".join([item.get_text(strip=True) for item in rent_info.select(".cassetteitem_price--rent, .cassetteitem_price--administration")]) if rent_info else None
    deposit_gratuity = " / ".join([price.get_text(strip=True) for price in rent_info.select(".cassetteitem_price--deposit, .cassetteitem_price--gratuity")]) if rent_info else None
    layout_total_area = " / ".join([detail.get_text(strip=True) for detail in rent_info.select(".cassetteitem_madori, .cassetteitem_menseki")]) if rent_info else None
    print("賃料/管理費 (Rent/Administration Fee):", rent_admin_fee)
    print("敷金/礼金 (Deposit/Key Money):", deposit_gratuity)
    print("間取り/占有面積 (Layout/Total Area):", layout_total_area)
except AttributeError as e:
    print("Error finding rent_info:", e)

物件名称 (Property Name): cocolo
カテゴリー (Category): 賃貸アパート
住所 (Address): 東京都大田区南六郷３
最寄り駅 (Nearest Stations): ['京急本線/雑色駅 歩8分', '京急本線/六郷土手駅 歩15分', '京急大師線/鈴木町駅 歩23分']
築年数 (Years Since Construction): 築11年
階建 (Number of Floors): 3階建
階数 (Floor Number): None
賃料/管理費 (Rent/Administration Fee): 7.4万円 / 3000円
敷金/礼金 (Deposit/Key Money): - / -
間取り/占有面積 (Layout/Total Area): 1SK / 25.38m2


In [16]:
# 物件画像・間取り画像・詳細URLの取得
property_image_element = items.find(class_="cassetteitem_object-item")
property_image_url = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

floor_plan_image_element = items.find(class_="casssetteitem_other-thumbnail")
floor_plan_image_url = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

property_link_element = items.select_one("a[href*='/chintai/jnc_']")
property_link = "*******" + property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

# 物件画像・間取り画像・詳細URLの表示
print("物件画像 URL (Property Image URL):", property_image_url)
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)
print("物件リンク (Property Link):", property_link)

物件画像 URL (Property Image URL): https://img01.suumo.com/front/gazo/fr/bukken/590/100382399590/100382399590_gw.jpg
間取り情報画像 URL (Floor Plan Image URL): https://img01.suumo.com/front/gazo/fr/bukken/590/100382399590/100382399590_co.jpg
物件リンク (Property Link): *******/chintai/jnc_000090729199/?bc=100382399590


### ● 不動産の複数ページから情報を取得する。

In [17]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13111&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
max_page = 5  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "******" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)    


page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30


In [18]:
df = pd.DataFrame(all_data)
df = df.drop_duplicates() # 重複データの削除
df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,cocolo,賃貸アパート,東京都大田区南六郷３,"京急本線/雑色駅 歩8分, 京急本線/六郷土手駅 歩15分, 京急大師線/鈴木町駅 歩23分",築11年,3階建,1-2階,7.4万円,3000円,-,-,1SK,25.38m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://img01.suumo.com/front/gazo/fr/bukken/5...,******/chintai/jnc_000090729199/?bc=100382399590
1,My Style vintage西六郷,賃貸マンション,東京都大田区西六郷２,"京急本線/雑色駅 歩9分, ＪＲ京浜東北線/蒲田駅 歩19分, 京急本線/六郷土手駅 歩18分",新築,3階建,2階,10.1万円,5000円,-,-,1SK,28.77m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/4...,******/chintai/jnc_000090702825/?bc=100382571449


### ● Googleスプレッドシートへの書き込み・読み込み。

In [19]:
# google スプレッドシート 書き込み・読み込み
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [20]:
from dotenv import load_dotenv
import os

# 環境変数の読み込み
load_dotenv()

# 環境変数から認証情報を取得
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
PRIVATE_KEY_PATH = os.getenv("PRIVATE_KEY_PATH")

In [21]:
PRIVATE_KEY_PATH

'C:\\Users\\yuya.hosokawa\\Documents\\STEP3\\STEP3-1_bady\\grspread_key.json'

In [22]:
# googleスプレッドシートの認証 jsonファイル読み込み(key値はGCPから取得)
SP_CREDENTIAL_FILE = PRIVATE_KEY_PATH

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    SP_CREDENTIAL_FILE,
    scopes=scopes
)
gc = gspread.authorize(credentials)


SP_SHEET_KEY = SPREADSHEET_ID # d/〇〇/edit の〇〇部分
sh  = gc.open_by_key(SP_SHEET_KEY)

In [23]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_90' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, df)

In [49]:
# 不動産データの取得
SP_SHEET     = 'tech0_90' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [50]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,cocolo,賃貸アパート,東京都大田区南六郷３,"京急本線/雑色駅 歩8分, 京急本線/六郷土手駅 歩15分, 京急大師線/鈴木町駅 歩23分",築11年,3階建,1-2階,7.4万円,3000円,-,-,1SK,25.38m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,https://img01.suumo.com/front/gazo/fr/bukken/5...,******/chintai/jnc_000090729199/?bc=100382399590
1,My Style vintage西六郷,賃貸マンション,東京都大田区西六郷２,"京急本線/雑色駅 歩9分, ＪＲ京浜東北線/蒲田駅 歩19分, 京急本線/六郷土手駅 歩18分",新築,3階建,2階,10.1万円,5000円,-,-,1SK,28.77m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/4...,******/chintai/jnc_000090702825/?bc=100382571449


### ● 不動産データを加工する。

In [51]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   名称       72 non-null     object
 1   カテゴリ     72 non-null     object
 2   アドレス     72 non-null     object
 3   アクセス     72 non-null     object
 4   築年数      72 non-null     object
 5   構造       72 non-null     object
 6   階数       72 non-null     object
 7   家賃       72 non-null     object
 8   管理費      72 non-null     object
 9   敷金       72 non-null     object
 10  礼金       72 non-null     object
 11  間取り      72 non-null     object
 12  面積       72 non-null     object
 13  物件画像URL  72 non-null     object
 14  間取画像URL  72 non-null     object
 15  物件詳細URL  72 non-null     object
dtypes: object(16)
memory usage: 9.1+ KB


In [52]:
new_df['築年数'] = new_df["築年数"].apply( lambda x: 0 if x=='新築' else int(re.split('[築年]', x )[1]) )

In [53]:
def get_most_floor(x):
    if ('階建' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階建',str(x))
        list = map(int, list)
        min_value = min(list)
        return min_value

new_df['構造'] = new_df['構造'].apply(get_most_floor)
print(new_df['構造'].head(5))

0    3
1    3
2    3
3    3
4    3
Name: 構造, dtype: int64


In [54]:
def get_floor(x):
    if ('階' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = min(list)
        return min_value
    else:
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = -1*min(list)
        return min_value

new_df['階数'] = new_df['階数'].apply(get_floor)
print(new_df['階数'].head(5))

0    2.0
1    2.0
2    3.0
3    3.0
4    2.0
Name: 階数, dtype: float64


In [55]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

new_df['家賃'] = new_df['家賃'].apply(change_fee)
new_df['敷金'] = new_df['敷金'].apply(change_fee)
new_df['礼金'] = new_df['礼金'].apply(change_fee)

In [56]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


new_df['管理費'] = new_df['管理費'].apply(change_fee2)

In [57]:
new_df['面積'] = new_df['面積'].apply(lambda x: float(x[:-2]))

In [58]:
new_df['区'] = new_df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [59]:
new_df['市町'] = new_df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

In [60]:
def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = access.split('/')
        if len(parts) == 2:
            line_station, walk = parts
            # ' 歩'で分割できるか確認
            if ' 歩' in walk:
                station, walk_min = walk.split(' 歩')
                # 歩数の分の数値だけを抽出
                walk_min = int(re.search(r'\d+', walk_min).group())
            else:
                station = None
                walk_min = None
        else:
            line_station = access
            station = walk_min = None

        results[f'アクセス①{i}線路名'] = line_station
        results[f'アクセス①{i}駅名'] = station
        results[f'アクセス①{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
new_df = new_df.join(new_df.apply(split_access, axis=1))

In [61]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,...,市町,アクセス①1線路名,アクセス①1駅名,アクセス①1徒歩(分),アクセス①2線路名,アクセス①2駅名,アクセス①2徒歩(分),アクセス①3線路名,アクセス①3駅名,アクセス①3徒歩(分)
0,cocolo,賃貸アパート,東京都大田区南六郷３,"京急本線/雑色駅 歩8分, 京急本線/六郷土手駅 歩15分, 京急大師線/鈴木町駅 歩23分",11,3,2.0,7.4,3000.0,NaN,...,南六郷,京急本線,雑色駅,8,京急本線,六郷土手駅,15,京急大師線,鈴木町駅,23
1,My Style vintage西六郷,賃貸マンション,東京都大田区西六郷２,"京急本線/雑色駅 歩9分, ＪＲ京浜東北線/蒲田駅 歩19分, 京急本線/六郷土手駅 歩18分",0,3,2.0,10.1,5000.0,NaN,...,西六郷,京急本線,雑色駅,9,ＪＲ京浜東北線,蒲田駅,19,京急本線,六郷土手駅,18


In [62]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   名称           72 non-null     object 
 1   カテゴリ         72 non-null     object 
 2   アドレス         72 non-null     object 
 3   アクセス         72 non-null     object 
 4   築年数          72 non-null     int64  
 5   構造           72 non-null     int64  
 6   階数           71 non-null     float64
 7   家賃           72 non-null     float64
 8   管理費          69 non-null     float64
 9   敷金           49 non-null     float64
 10  礼金           52 non-null     float64
 11  間取り          72 non-null     object 
 12  面積           72 non-null     float64
 13  物件画像URL      72 non-null     object 
 14  間取画像URL      72 non-null     object 
 15  物件詳細URL      72 non-null     object 
 16  区            72 non-null     object 
 17  市町           72 non-null     object 
 18  アクセス①1線路名    72 non-null     object 
 19  アクセス①1駅名  

In [63]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_91' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)

### ● 緯度経度情報取得

In [68]:
# 不動産データの取得
SP_SHEET     = 'tech0_91' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [69]:
new_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   名称           72 non-null     object
 1   カテゴリ         72 non-null     object
 2   アドレス         72 non-null     object
 3   アクセス         72 non-null     object
 4   築年数          72 non-null     object
 5   構造           72 non-null     object
 6   階数           72 non-null     object
 7   家賃           72 non-null     object
 8   管理費          72 non-null     object
 9   敷金           72 non-null     object
 10  礼金           72 non-null     object
 11  間取り          72 non-null     object
 12  面積           72 non-null     object
 13  物件画像URL      72 non-null     object
 14  間取画像URL      72 non-null     object
 15  物件詳細URL      72 non-null     object
 16  区            72 non-null     object
 17  市町           72 non-null     object
 18  アクセス①1線路名    72 non-null     object
 19  アクセス①1駅名     72 non-null     ob

In [70]:
# ジオコーダーの初期化
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="your_app_name")

current_count = 0
total_count = len(new_df['アドレス'])


# 住所から緯度と経度を取得する関数
def get_lat_lon(address):
    global current_count
    current_count += 1

    try:
        location = geolocator.geocode(address)
        print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")

        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving location for address {address}: {e}")
        return None, None

In [71]:
location = geolocator.geocode("東京都江戸川区篠崎町７")
print(location)

None


In [73]:
# 住所列があると仮定して、緯度経度列をデータフレームに追加
new_df['latitude'], new_df['longitude'] = zip(*new_df['アドレス'].apply(get_lat_lon))

Error retrieving location for address 東京都大田区南六郷３: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区西六郷２: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区西六郷２: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区西六郷２: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区西六郷２: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区西六郷２: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区西六郷２: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区山王１: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都大田区山王１: 'NoneType' object has no attribute 'latitude'
82/72 件目実施中 結果: 35.5616494, 139.722177
Error retrieving location for address 東京都大田区大森東２: 'NoneType' object has no attribute 'latitude'
Err

In [74]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_01' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)